In [ ]:
import sqlite3
import time
import pymongo
import numpy as np
import json
import requests
import pandas as pd
import datetime
from PIL import Image
from IPython.display import HTML
from matplotlib.image import imread
from pandasql import sqldf
import matplotlib.pyplot as plt
%matplotlib inline

In [142]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

2011-07-15    0
2011-07-16    0
2011-07-17    1
2011-07-18    1
2011-07-19    0
2011-07-20    0
2011-07-21    1
2011-07-22    1
2011-07-23    1
2011-07-24    0
2011-07-25    0
2011-07-26    0
2011-07-27    0
2011-07-28    1
2011-07-29    1
2011-07-30    1
2011-07-31    1
2011-08-01    1
2011-08-02    1
2011-08-03    0
2011-08-04    1
2011-08-05    1
2011-08-06    1
2011-08-07    1
2011-08-08    1
2011-08-09    0
2011-08-10    1
2011-08-11    1
2011-08-12    1
2011-08-13    0
             ..
2012-04-07    0
2012-04-08    0
2012-04-09    0
2012-04-10    0
2012-04-11    0
2012-04-12    0
2012-04-13    0
2012-04-14    0
2012-04-15    0
2012-04-16    0
2012-04-17    0
2012-04-18    0
2012-04-19    0
2012-04-20    0
2012-04-21    0
2012-04-22    0
2012-04-23    0
2012-04-24    1
2012-04-25    1
2012-04-26    0
2012-04-27    0
2012-04-28    0
2012-04-29    0
2012-04-30    0
2012-05-01    0
2012-05-02    0
2012-05-03    1
2012-05-04    0
2012-05-05    1
2012-05-06    1
Name: 0, Length: 297, dt

In [ ]:
c.execute("""SELECT *
             FROM Matches
             WHERE Season IN (2011) AND Div IN ('D1','D2') 
             ORDER BY Date
             """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

In [ ]:
dict_of_win = {}
for i in list(df.AwayTeam.unique()):
    dict_of_win.update({i: 0})
for n in range(len(df)):
    if df.FTR.iloc[n] == 'H':
        dict_of_win[df.HomeTeam.iloc[n]] += 1
    elif df.FTR.iloc[n] == 'A':
        dict_of_win[df.AwayTeam.iloc[n]] += 1

In [ ]:
dict_of_loss = {}
for i in list(df.AwayTeam.unique()):
    dict_of_loss.update({i: 0})
for n in range(len(df)):
    if df.FTR.iloc[n] == 'H':
        dict_of_loss[df.AwayTeam.iloc[n]] += 1
    elif df.FTR.iloc[n] == 'A':
        dict_of_loss[df.HomeTeam.iloc[n]] += 1

In [ ]:
dict_of_draw = {}
for i in list(df.AwayTeam.unique()):
    dict_of_draw.update({i: 0})
for n in range(len(df)):
    if df.FTR.iloc[n] == 'D':
        dict_of_draw[df.AwayTeam.iloc[n]] += 1
        dict_of_draw[df.HomeTeam.iloc[n]] += 1

In [ ]:
dates = [str(datetime.date(2011,7, 15) + datetime.timedelta(i)) for i in range(297)]

#time set at 3pm GMT to find 4pm European Central Time
# urls = ["https://api.darksky.net/forecast/520ec0a7ffac4abd104235b232a81fbb/52.5200,13.4050,"\
#                        +date+"T15:00:00Z?exclude=currently,hourly,flags" for date in dates]
#retrieve data and unstring
data = []
for i in urls:
    data.append(json.loads(requests.get(i).text))
#unnest dictionaries
daily_data = [data[i]['daily']['data'][0] for i in range(len(data))]

#extract precipitation type
precipType = []
for i in range(len(daily_data)):
    if 'precipType' in daily_data[i].keys():
        precipType.append(daily_data[i]['precipType'])
    else:
        precipType.append('0')

#Create dataframe with dates and precipitation type
weatherdf = pd.DataFrame(list(zip(dates, precipType)), 
               columns =['Date', 'Rain'])
#change strings into rain dummy variable
weatherdf['Rain']= weatherdf['Rain'].map({'0':0, 'rain':1, 'sleet':0, 'snow':0})



In [ ]:
df = df.merge(weatherdf, on='Date', how='left')
raindf = df[df['Rain']==1]

dict_of_win_rain = {}
for i in list(raindf.AwayTeam.unique()):
    dict_of_win_rain.update({i: 0})
for n in range(len(raindf)):
    if raindf.FTR.iloc[n] == 'H':
        dict_of_win_rain[raindf.HomeTeam.iloc[n]] += 1
    elif df.FTR.iloc[n] == 'A':
        dict_of_win_rain[raindf.AwayTeam.iloc[n]] += 1

dict_of_loss_rain = {}
for i in list(raindf.AwayTeam.unique()):
    dict_of_loss_rain.update({i: 0})
for n in range(len(raindf)):
    if df.FTR.iloc[n] == 'H':
        dict_of_loss_rain[raindf.AwayTeam.iloc[n]] += 1
    elif df.FTR.iloc[n] == 'A':
        dict_of_loss_rain[raindf.HomeTeam.iloc[n]] += 1

dict_of_draw_rain = {}
for i in list(raindf.AwayTeam.unique()):
    dict_of_draw_rain.update({i: 0})
for n in range(len(raindf)):
    if raindf.FTR.iloc[n] == 'D':
        dict_of_draw_rain[raindf.AwayTeam.iloc[n]] += 1
        dict_of_draw_rain[raindf.HomeTeam.iloc[n]] += 1

In [ ]:
finaldf = pd.DataFrame()
finaldf['Team'] = dict_of_win.keys()
finaldf['Wins_Overall'] = finaldf.Team.apply(lambda team: dict_of_win[team])
finaldf['Wins_Rain'] = finaldf.Team.apply(lambda team: dict_of_win_rain[team])
finaldf['Losses_Rain'] = finaldf.Team.apply(lambda team: dict_of_loss_rain[team])
finaldf['Draws_Rain'] = finaldf.Team.apply(lambda team: dict_of_draw_rain[team])
finaldf['Rain_Games'] = finaldf['Wins_Rain']+finaldf['Losses_Rain']+finaldf['Draws_Rain']
finaldf['No_Rain_Games'] = 34 - finaldf['Rain_Games']
finaldf['Wins_No_Rain'] = finaldf['Wins_Overall']-finaldf['Wins_Rain']
finaldf['Rain_Win%'] = finaldf['Wins_Rain']/finaldf['Rain_Games']
finaldf['No_Rain_Win%'] = finaldf['Wins_No_Rain']/finaldf['No_Rain_Games']
finaldf['%Change_Rain_Win_Rate'] = (finaldf['Rain_Win%']- finaldf['No_Rain_Win%'])/finaldf['No_Rain_Win%']
finaldf.head()